### Twogram Threegram Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# sentence check
sentence_check = True  # True, False
#twogram_sentence_check = False  # True, False
#threegram_sentence_check = False

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/1-Twogram Threegram Shared Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [8]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [10]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [11]:
word_list = list(word_select_set.difference(disable_word_set))

In [12]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_{data_kind.capitalize()}_Result{file_ext}.xlsx", index=False)    

#### Concat Result

In [13]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,gerçek
1,her
2,ister
3,sen
4,bu
...,...
195,ki
196,size
197,hey
198,bunun


In [14]:
# Option For Frequency
if shared_word_frequency:
    #df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
    df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
    df_word.drop_duplicates(inplace=True)
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    pass

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [15]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Result{file_ext}.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
25298,size ama,6
25299,şunu küçük,6
25300,yok kadar,6
25301,dakika ile,6


In [16]:
# Option For Twogram Sentence
#if twogram_sentence_check:
if sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
    df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
    df_shared_twogram.drop_duplicates(inplace=True)
    df_shared_twogram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_twogram

,twogram,freq_twogram
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
7598,aslında işte,6
7599,siz tabii,6
7600,bizim hemen,6
7601,aslında üzgünüm,6


In [17]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_Result{file_ext}.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,başka bir şey,75193
4,bir şey değil,68498
...,...,...
109657,peki ne şekilde,10
109658,şey ki çok,10
109659,peki neden olduğunu,10
109660,gibi size doğru,10


In [18]:
# Option For Threegram Sentence
#if threegram_sentence_check:
if sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram_sent = df_threegram_sent.loc[:,["threegram"]]
    df_shared_threegram = pd.merge(df_shared_threegram, df_threegram_sent, how="inner", on="threegram")
    df_shared_threegram.drop_duplicates(inplace=True)
    df_shared_threegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_threegram

,threegram,freq_threegram
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,başka bir şey,75193
4,bir şey değil,68498
...,...,...
11908,bu aslında doğru,10
11909,bu sadece biri,10
11910,peki git ve,10
11911,bu saat benim,10


In [19]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram], axis=1)
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram
0,bir,18835735.0,bir şey,859944.0,bir şey yok,113165
1,bu,11062659.0,değil mi,585879.0,bir şey var,110455
2,ne,8025880.0,ben de,377765.0,bu da ne,89463
3,ve,7766036.0,teşekkür ederim,370619.0,başka bir şey,75193
4,için,5484109.0,ne oldu,322758.0,bir şey değil,68498
...,...,...,...,...,...,...
11908,NaN,NaN,NaN,NaN,bu aslında doğru,10
11909,NaN,NaN,NaN,NaN,bu sadece biri,10
11910,NaN,NaN,NaN,NaN,peki git ve,10
11911,NaN,NaN,NaN,NaN,bu saat benim,10


In [20]:
# Optional Output
#if sentence_check:
#    df_shared_all.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Threegram_Sentence_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_twogram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Sentence_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_threegram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_Sentence_Result_With_Frequency{file_ext}.xlsx", index=False)
#else:
#    df_shared_all.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Threegram_All_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_twogram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_All_Result_With_Frequency{file_ext}.xlsx", index=False)
#    df_shared_threegram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_All_Result_With_Frequency{file_ext}.xlsx", index=False)

##### Concat Result With Comma

In [21]:
# \******
df_word_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
df_word_prefix_suffix = df_word_prefix_suffix[["search_word","word"]]
df_word_prefix_suffix

,search_word,word
0,adam,adam
1,adam,adama
2,adam,adamdan
3,adam,adamdı
4,adam,adamdır
...,...,...
1618,şey,şeyler
1619,şey,şeylerden
1620,şey,şeylere
1621,şey,şeyleri


In [22]:
if file_ext == "5":
    df_all_word = df_word_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "6":
    df_word_not_prefix_suffix = df_word_all
    df_word_not_prefix_suffix["search_word"] = df_word_all["word"]
    df_word_not_prefix_suffix = df_word_not_prefix_suffix[["search_word","word"]]

    df_all_word = df_word_not_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
else:
    pass
    
df_all_word
# /******

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,search_word,word
0,bir,bir
1,bu,bu
2,ne,ne
3,ve,ve
4,için,için
...,...,...
987924,karneleme,karneleme
987925,karnaya,karnaya
987926,dörtlümüzün,dörtlümüzün
987927,karnavalınız,karnavalınız


In [23]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [24]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram")  

In [25]:
# \*****
df_word_order_twogram = pd.merge(df_word_order_twogram,df_all_word, how="inner", on="word")
df_word_order_threegram = pd.merge(df_word_order_threegram,df_all_word, how="inner", on="word")

In [26]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["search_word"])["twogram"].apply(", ".join).reset_index()  
df_word_order_join_threegram = df_word_order_threegram.groupby(["search_word"])["threegram"].apply(", ".join).reset_index()

In [27]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram]

In [28]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['search_word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all.rename(columns={"search_word":"word"}, inplace=True)
df_word_order_join_all 

,word,twogram,threegram
0,adam,"bir adam, bu adam, o adam, adam var, adam mı, ...","bir adam var, iyi bir adam, bir adam vardı, bu..."
1,al,"al bakalım, al şunu, al bunu, al onu, al sana,...","al ve git, gel de al, al o zaman, al senin ols..."
2,ama,"hadi ama, ama bu, ama ben, evet ama, ama o, am...","ama yine de, çok ama çok, ama bir şey, ama değ..."
3,anne,"merhaba anne, selam anne, hayır anne, evet ann...","değil mi anne, anne iyi misin, anne ve baba, t..."
4,artık,"yok artık, ama artık, artık çok, artık değil, ...","artık daha fazla, belki de artık, artık hiçbir..."
...,...,...,...
195,şeyi,"her şeyi, bir şeyi, aynı şeyi, hiçbir şeyi, bu...","böyle bir şeyi, bir şeyi yok, her şeyi biliyor..."
196,şeyler,"bir şeyler, şeyler var, böyle şeyler, kötü şey...","bir şeyler var, bir şeyler oluyor, bir şeyler ..."
197,şimdi,"şimdi ne, şimdi de, şimdi mi, ama şimdi, ve şi...","şimdi ne olacak, ve şimdi de, ne demek şimdi, ..."
198,şu,"şu ki, ama şu, ve şu, gerçek şu, ver şu, sorun...","gerçek şu ki, sorun şu ki, bir de şu, peki ya ..."


In [29]:
df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
df_word_order_join_all.drop_duplicates(inplace=True)
df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram"]]
df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
df_word_order_join_all.reset_index(drop=True, inplace=True)
df_word_order_join_all

,word,frequency,twogram,threegram
0,bir,18835735,"bir şey, bu bir, bir dakika, bir şeyler, başka...","bir şey yok, bir şey var, başka bir şey, bir ş..."
1,bu,11062659,"bu kadar, bu da, bu çok, bu bir, bu ne, bu yüz...","bu da ne, neden bu kadar, bu doğru değil, bu i..."
2,ne,8025880,"ne oldu, ne kadar, ne yapıyorsun, ne demek, ne...","bu da ne, ne demek istiyorsun, burada ne yapıy..."
3,ve,7766036,"ve ben, ve bu, ve bir, sen ve, ve sen, ve o, v...","sen ve ben, ve ben de, ve sen de, ve bu da, ve..."
4,için,5484109,"senin için, benim için, bunun için, ne için, i...","benim için bir, bu senin için, her şey için, b..."
...,...,...,...,...
195,saat,399989,"bir saat, birkaç saat, iki saat, saat daha, üç...","bir saat içinde, bir saat önce, bir saat sonra..."
196,onunla,399330,"onunla ne, ben onunla, onunla birlikte, onunla...","ben de onunla, onunla ne yapıyorsun, ve onunla..."
197,yapıyorsun,398274,"ne yapıyorsun, neden yapıyorsun, bunu yapıyors...","burada ne yapıyorsun, ne yapıyorsun sen, sen n..."
198,neler,397377,"neler oluyor, neler olduğunu, burada neler, ne...","burada neler oluyor, neler oluyor burada, orad..."


In [30]:
def lower_strip_func(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [31]:
df_word_all = df_word_all.loc[:,["word","frequency"]]
df_word_all["word"] = df_word_all["word"].apply(lambda x: lower_strip_func(x))
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [32]:
df_twogram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_all = df_twogram_all.loc[:,["twogram","frequency"]]
df_twogram_all["twogram"] = df_twogram_all["twogram"].apply(lambda x: lower_strip_func(x))
df_twogram_all

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [33]:
df_threegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_threegram_all = df_threegram_all.loc[:,["threegram","frequency"]]
df_threegram_all["threegram"] = df_threegram_all["threegram"].apply(lambda x: lower_strip_func(x))
df_threegram_all

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [34]:
df_shared_process_all = df_word_order_join_all
for i in range(len(df_shared_process_all)):
    # column result
    try:
        # column result
        df_two_var = pd.DataFrame(df_shared_process_all.loc[i,"twogram"].split(", "), columns=["twogram"])
        # merge with all
        df_two_var_merge = pd.merge(df_two_var, df_twogram_all, how="left", on="twogram")
        df_two_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_two_var_merge_select = df_two_var_merge.head(word_sample_num)
        # to list value
        df_two_var_list = df_two_var_merge_select["twogram"].to_list()
        # list join
        df_two_var_list_join = ", ".join(df_two_var_list)
        # change value
        df_shared_process_all.loc[i,"twogram"] = df_two_var_list_join
    except:
        pass
    try:
        # column result
        df_three_var = pd.DataFrame(df_shared_process_all.loc[i,"threegram"].split(", "), columns=["threegram"])
        # merge with all
        df_three_var_merge = pd.merge(df_three_var, df_threegram_all, how="left", on="threegram")
        df_three_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_three_var_merge_select = df_three_var_merge.head(word_sample_num)
        # to list value
        df_three_var_list = df_three_var_merge_select["threegram"].to_list()
        # list join
        df_three_var_list_join = ", ".join(df_three_var_list)
        # change value
        df_shared_process_all.loc[i,"threegram"] = df_three_var_list_join
    except:
        pass

df_shared_process_all
# /****

,word,frequency,twogram,threegram
0,bir,18835735,"bir şey, bu bir, bir dakika, bir şeyler, başka...","bir şey yok, bir şey var, başka bir şey, bir ş..."
1,bu,11062659,"bu kadar, bu da, bu çok, bu bir, bu ne, bu yüz...","bu da ne, neden bu kadar, bu doğru değil, bu i..."
2,ne,8025880,"ne oldu, ne kadar, ne yapıyorsun, ne demek, ne...","bu da ne, ne demek istiyorsun, burada ne yapıy..."
3,ve,7766036,"ve ben, ve bu, ve bir, sen ve, ve sen, ve o, v...","sen ve ben, ve ben de, ve sen de, ve bu da, ve..."
4,için,5484109,"senin için, benim için, bunun için, ne için, i...","benim için bir, bu senin için, her şey için, b..."
...,...,...,...,...
195,saat,399989,"bir saat, birkaç saat, iki saat, saat daha, üç...","bir saat içinde, bir saat önce, bir saat sonra..."
196,onunla,399330,"onunla ne, ben onunla, onunla birlikte, onunla...","ben de onunla, onunla ne yapıyorsun, ve onunla..."
197,yapıyorsun,398274,"ne yapıyorsun, neden yapıyorsun, bunu yapıyors...","burada ne yapıyorsun, ne yapıyorsun sen, sen n..."
198,neler,397377,"neler oluyor, neler olduğunu, burada neler, ne...","burada neler oluyor, neler oluyor burada, orad..."


In [35]:
df_word_order_join_twogram = df_shared_process_all[["word","frequency","twogram"]]
df_word_order_join_threegram = df_shared_process_all[["word","frequency","threegram"]]

In [36]:
if sentence_check:
    df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Threegram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_twogram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_threegram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_Sentence_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
else:
    df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_Threegram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_twogram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Twogram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)
    df_word_order_join_threegram.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Threegram_All_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [37]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word*{file_ext}.xlsx")
output_file

['Turkish_200_Word_Twogram_Result6.xlsx',
 'Turkish_200_Word_Threegram_Result6.xlsx',
 'Turkish_200_Word_Twogram_Threegram_Sentence_Join_Result_Without_Frequency6.xlsx',
 'Turkish_200_Word_Twogram_Sentence_Join_Result_Without_Frequency6.xlsx',
 'Turkish_200_Word_Threegram_Sentence_Join_Result_Without_Frequency6.xlsx']

In [38]:
for k in output_file:
    source = k # source directory
    #destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    destination = path
    shutil.copy2(source, destination)

In [39]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass